### A full training

In [1]:
import torch
import numpy as np

torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

现在，我们将看到如何在不使用Trainer类的情况下实现与上一节相同的结果。同样，我们假设您已经完成了第2节中的数据处理。这里有一个简短的总结，涵盖了你需要的一切:

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (C:/Users/Jager/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Jager\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-5137f2fb4737d116.arrow
Loading cached processed dataset at C:\Users\Jager\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-60ba0ce5ca5c6a38.arrow
Loading cached processed dataset at C:\Users\Jager\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-b26904918f30dd5d.arrow


### Prepare for training

在实际编写训练循环之前，我们需要定义一些对象。第一个是我们将用于迭代批处理的数据加载器。但在定义这些数据加载器之前，我们需要对tokenized_datasets进行一些后处理，让Trainer自动为我们做一些事情。具体来说，我们需要:
- 删除模型中不需要使用的列(如sentence1和sentence2列)
- 将列标签重命名为labels(因为模型期望参数被命名为labels)
- 设置数据集的格式，使它们返回PyTorch张量而不是列表
我们的tokenized_datasets对于每一个步骤都有一个对应的方法:

In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

现在，我们可以很容易地定义我们的数据加载器:

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

为了快速检查数据处理没有错误，我们可以这样检查一批数据:

In [5]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 61]),
 'token_type_ids': torch.Size([8, 61]),
 'attention_mask': torch.Size([8, 61])}

请注意，实际的形状可能会略有不同，因为我们为训练数据加载器设置了shuffle=True，并且我们在批处理中填充到最大长度。
现在我们已经完成了数据预处理(对于任何ML从业者来说，这是一个令人满意但难以实现的目标)，让我们转向模型。我们像上一节一样实例化它:

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

为了确保在训练过程中一切顺利，我们将批处理传递给这个模型:

In [7]:
outputs = model(**batch)
outputs.loss, outputs.logits.shape

(tensor(0.69, grad_fn=<NllLossBackward0>), torch.Size([8, 2]))

当提供标签时，所有Transformer模型都将返回损失，并且我们还获得logits(批处理中的每个输入两个，因此大小为8 x 2的张量)。

我们几乎已经准备好编写训练循环了!我们只缺少两样东西:一个优化器和一个学习率调度器。因为我们正在手动复现Trainer做的活，这里使用相同的默认值。
训练器使用的优化器是AdamW，它与Adam相同，但对权重衰减正则化有所改变(参见Ilya Loshchilov和Frank Hutter的“解耦权重衰减正则化”):

In [9]:
from transformers import AdamW

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

最后，默认情况下使用的学习率调度器只是从最大值(5e-5)到0的线性衰减。为了正确地定义它，需要知道采取的训练步骤的数量，这是我们想要运行的epoch的数量乘以训练批次的数量(这是我们的训练数据加载器的长度)。Trainer默认使用三个epoch，所以我们将遵循它:

In [10]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


In [ ]:
### The training loop

最后一件事:如果我们有机会使用GPU(在CPU上，训练可能需要几个小时而不是几分钟)，我们将希望使用GPU。为了做到这一点，我们定义了一个设备，我们将把我们的模型和批次放在上面:

In [11]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

我们现在准备好训练了!为了了解训练何时完成，我们使用tqdm库在训练步骤数上添加进度条:

In [12]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

您可以看到，训练循环的核心看起来很像介绍中的那个。我们没有要求任何报告，所以这个训练循环不会告诉我们任何关于模型如何运行的信息。我们需要为它添加一个求值循环。

In [ ]:
### The evaluation loop

正如前面所做的，我们将使用Evaluate库提供的一个度量。我们已经看到了metric.compute()方法，但是当我们使用add_batch()方法遍历预测循环时，metrics实际上可以为我们累积批。一旦我们累积了所有批次，我们就可以使用metric.compute()获得最终结果。下面是如何在求值循环中实现所有这些:

In [13]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8529411764705882, 'f1': 0.8951048951048952}

同样，由于模型头部初始化和数据洗牌的随机性，您的结果将略有不同，但它们应该在相同的范围内。